# EDA — Service Classification Diagnosis (ODS6)

**Goal:** Diagnose why the `AAA` (Water + Sewerage + Cleaning) KPI was showing `0.0%` even though there is visible data suggesting such services are present.

This notebook explores the column `SERVICIO`, generates per-row service flags, reconstructs classification, and recalculates KPIs.


In [1]:
import pandas as pd
import sqlite3

DB_PATH = "../database/rups.db"
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql("SELECT * FROM prestadores", conn)
print("Rows:", len(df))
df.head()


Rows: 9286


,DEPARTAMENTO_PRESTACION,MUNICIPIO_PRESTACION,SERVICIO,ESTADO,NOMBRE,NIT,DEPARTAMENTO_DOMICILIO,MUNICIPIO_DOMICILIO,DIRECCION,TELEFONO,EMAIL,TIPO_INSCRIPCION,REPRESENTANTE_LEGAL,TIPO_PRESTADOR,CLASIFICACION
0,CALDAS,RIOSUCIO,ASEO,OPERATIVA,EMPRESA MUNICIPAL DE SERVICIOS DE ASEO DE RIOS...,890801631-4,CALDAS,RIOSUCIO,CARRERA 7 CALLE 10 ESQUINA 1 PISO PALACIO MUNI...,8592201.0,controlinterno@emsaesp.gov.co,Registro por la ESP,OLGA LUCIA CASTANO BENJUMEA,EMPRESA INDUSTRIAL Y COMERCIAL DEL ESTADO,MAYOR O IGUAL A 5001 USUARIOS
1,CALDAS,RISARALDA,ASEO,OPERATIVA,OFICINA DE DESARROLLO ECONOMICO Y SERVICIOS PU...,800095461-1,CALDAS,RISARALDA,Carrera 2 Calle 5 Esquina,8557220.0,alcaldia@risaralda-caldas.gov.co,Registro por la ESP,JORGE DANILO GUTIERREZ CUARTAS,MUNICIPIO (PRESTACIÓN DIRECTA),HASTA 2500 SUSCRIPTORES
2,CALDAS,NEIRA,ASEO,OPERATIVA,AQUAMANA E.S.P.,810001898-1,CALDAS,VILLAMARIA,CALLE 9 Nº 4-29,8775141.0,aagcsis@gmail.com,Registro por la ESP,JUAN PABLO BUITRAGO QUICENO,EMPRESA INDUSTRIAL Y COMERCIAL DEL ESTADO,DESDE 2501 HASTA 5000 USUARIOS
3,CALDAS,VILLAMARIA,ASEO,OPERATIVA,AQUAMANA E.S.P.,810001898-1,CALDAS,VILLAMARIA,CALLE 9 Nº 4-29,8775141.0,aagcsis@gmail.com,Registro por la ESP,JUAN PABLO BUITRAGO QUICENO,EMPRESA INDUSTRIAL Y COMERCIAL DEL ESTADO,DESDE 2501 HASTA 5000 USUARIOS
4,CALDAS,VILLAMARIA,ACUEDUCTO,OPERATIVA,AQUAMANA E.S.P.,810001898-1,CALDAS,VILLAMARIA,CALLE 9 Nº 4-29,8775141.0,aagcsis@gmail.com,Registro por la ESP,JUAN PABLO BUITRAGO QUICENO,EMPRESA INDUSTRIAL Y COMERCIAL DEL ESTADO,MAYOR O IGUAL A 5001 USUARIOS


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9286 entries, 0 to 9285
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DEPARTAMENTO_PRESTACION  9286 non-null   object 
 1   MUNICIPIO_PRESTACION     9286 non-null   object 
 2   SERVICIO                 9286 non-null   object 
 3   ESTADO                   9286 non-null   object 
 4   NOMBRE                   9286 non-null   object 
 5   NIT                      9286 non-null   object 
 6   DEPARTAMENTO_DOMICILIO   9194 non-null   object 
 7   MUNICIPIO_DOMICILIO      9194 non-null   object 
 8   DIRECCION                9191 non-null   object 
 9   TELEFONO                 9194 non-null   float64
 10  EMAIL                    9194 non-null   object 
 11  TIPO_INSCRIPCION         9286 non-null   object 
 12  REPRESENTANTE_LEGAL      9277 non-null   object 
 13  TIPO_PRESTADOR           8820 non-null   object 
 14  CLASIFICACION           

In [3]:
(df.isna().mean() * 100).round(1).sort_values(ascending=False)

CLASIFICACION              5.1
TIPO_PRESTADOR             5.0
DEPARTAMENTO_DOMICILIO     1.0
DIRECCION                  1.0
MUNICIPIO_DOMICILIO        1.0
TELEFONO                   1.0
EMAIL                      1.0
REPRESENTANTE_LEGAL        0.1
NOMBRE                     0.0
NIT                        0.0
SERVICIO                   0.0
DEPARTAMENTO_PRESTACION    0.0
MUNICIPIO_PRESTACION       0.0
ESTADO                     0.0
TIPO_INSCRIPCION           0.0
dtype: float64

In [21]:
df.describe(include="all")


,DEPARTAMENTO_PRESTACION,MUNICIPIO_PRESTACION,SERVICIO,ESTADO,NOMBRE,NIT,DEPARTAMENTO_DOMICILIO,MUNICIPIO_DOMICILIO,DIRECCION,TELEFONO,EMAIL,TIPO_INSCRIPCION,REPRESENTANTE_LEGAL,TIPO_PRESTADOR,CLASIFICACION,has_acueducto,has_alcantarillado,has_aseo,service_class
count,9286,9286,9286,9286,9286,9286,9194,9194,9191,9.194000e+03,9194,9286,9277,8820,8814,9286.000000,9286.000000,9286.000000,9286
unique,33,1018,3,5,4151,4154,33,988,4019,NaN,3640,1,4070,6,5,NaN,NaN,NaN,3
top,ANTIOQUIA,"BOGOTA, D.C.",ASEO,OPERATIVA,URBASER TUNJA S.A. E.S.P.,900159283-6,ANTIOQUIA,"BOGOTA, D.C.",TRANSVERSAL 15 No. 24 - 12,NaN,Luz.galvis@urbaser.co,Registro por la ESP,JUAN MANUEL GOMEZ MEJIA,ORGANIZACION AUTORIZADA,MENOR O IGUAL A 2500 USUARIOS,NaN,NaN,NaN,Cleaning only
freq,1179,384,4597,8783,112,112,1175,664,112,NaN,146,9286,141,3668,3600,NaN,NaN,NaN,4597
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.689915e+06,NaN,NaN,NaN,NaN,NaN,0.349128,0.155826,0.495046,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.566285e+06,NaN,NaN,NaN,NaN,NaN,0.476720,0.362710,0.500002,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.368040e+06,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.365067e+06,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.784069e+06,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,1.000000,NaN


Análisis bivariado (correlación cruzada de flags)

In [23]:
df[["has_acueducto", "has_alcantarillado", "has_aseo"]].corr()


,has_acueducto,has_alcantarillado,has_aseo
has_acueducto,1.000000,-0.314665,-0.725173
has_alcantarillado,-0.314665,1.000000,-0.425404
has_aseo,-0.725173,-0.425404,1.000000


Outliers o casos anómalos

In [24]:
df.groupby("DEPARTAMENTO_PRESTACION")["has_acueducto"].mean().sort_values()


DEPARTAMENTO_PRESTACION
BOGOTA, D.C.                                                0.091146
CASANARE                                                    0.131250
META                                                        0.199405
NORTE DE SANTANDER                                          0.230126
QUINDIO                                                     0.237705
SANTANDER                                                   0.254386
LA GUAJIRA                                                  0.273585
ATLANTICO                                                   0.276860
GUAVIARE                                                    0.277778
CESAR                                                       0.278912
AMAZONAS                                                    0.285714
CORDOBA                                                     0.292793
VAUPES                                                      0.300000
BOLIVAR                                                     0.306569
CAQUETA   

### Top 30 unique values in `SERVICIO` column

In [4]:
df['SERVICIO'].value_counts(dropna=False).head(30)

SERVICIO
ASEO              4597
ACUEDUCTO         3242
ALCANTARILLADO    1447
Name: count, dtype: int64

### Generate per-row flags

In [6]:
serv = df["SERVICIO"].astype("string").str.upper().fillna("")

df["has_acueducto"] = serv.str.contains("ACUEDUCTO|AAA", na=False).astype("int8")
df["has_alcantarillado"] = serv.str.contains("ALCANTARILLADO|AAA", na=False).astype("int8")
df["has_aseo"] = serv.str.contains("ASEO|AAA", na=False).astype("int8")

In [7]:
def classify(a, al, aseo):
    if a and al and aseo: return "AAA (Water+Sewerage+Cleaning)"
    if a and al and not aseo: return "AA (Water + Sewerage)"
    if a and not al and aseo: return "Water + Cleaning"
    if not a and al and aseo: return "Sewerage + Cleaning"
    if a and not al and not aseo: return "Water only"
    if not a and al and not aseo: return "Sewerage only"
    if not a and not al and aseo: return "Cleaning only"
    return "No service"

df["service_class"] = df.apply(lambda r: classify(r["has_acueducto"], r["has_alcantarillado"], r["has_aseo"]), axis=1)
df["service_class"].value_counts()


service_class
Cleaning only    4597
Water only       3242
Sewerage only    1447
Name: count, dtype: int64

In [8]:
pct_AAA = round((df["service_class"] == "AAA (Water+Sewerage+Cleaning)").mean() * 100, 2)
print("Percentage of AAA services:", pct_AAA, "%")

Percentage of AAA services: 0.0 %


In [20]:
# 3) Group-level OR aggregation (no “cleaning” of rows, just grouping)
KEY = ["NIT", "NOMBRE", "DEPARTAMENTO_PRESTACION", "MUNICIPIO_PRESTACION"]
g = (
    df.groupby(KEY, dropna=False)[["has_acueducto", "has_alcantarillado", "has_aseo"]]
      .max()
      .reset_index()
)

# 4) KPI at GROUP level (true AAA coverage)
mask_AAA_group = (g["has_acueducto"] == 1) & (g["has_alcantarillado"] == 1) & (g["has_aseo"] == 1)
pct_AAA_groups = round(mask_AAA_group.mean() * 100, 2)

# 5) Optional KPI at ROW level (literal share of AAA rows by class label)
pct_AAA_rows = round((df["service_class"] == "AAA (Water+Sewerage+Cleaning)").mean() * 100, 2)

summary = {
    "rows_raw": len(df),
    "unique_providers_by_name": df["NOMBRE"].nunique(dropna=True),
    "departments_covered": df["DEPARTAMENTO_PRESTACION"].fillna("NO_DATA").nunique(),
    "municipalities_covered": df["MUNICIPIO_PRESTACION"].fillna("NO_DATA").nunique(),
    "pct_AAA_groups": pct_AAA_groups,  # AAA at provider+location level
    
}



In [25]:
from pathlib import Path

# Define the reports directory (adjust if needed)
PROJECT_ROOT = Path().resolve().parent  # or just Path().resolve() if you're in root
REPORTS_DIR = PROJECT_ROOT / "reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)


# Export KPI summary to CSV
pd.Series(summary, name="value").to_csv(
    REPORTS_DIR / "kpi_summary.csv", header=True, encoding="utf-8"
)

# Print summary to console
print("\n--- EDA KPI Summary ---")
for k, v in summary.items():
    print(f"{k}: {v}")






--- EDA KPI Summary ---
rows_raw: 9286
unique_providers_by_name: 4151
departments_covered: 33
municipalities_covered: 1018
pct_AAA_groups: 13.68


## Conclusion

The original `AAA` KPI was 13.8% because the field `SERVICIO` 

By propagating `AAA` to represent the presence of the three services, the true distribution of multi-service providers can be recovered.
